In [23]:
import os 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from numpy import mean
from numpy import std
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE


In [132]:
train = pd.read_csv('exprs_train.csv')
test = pd.read_csv('exprs_test.csv')

# EDA y Preprocesamiento de Datos

In [133]:
train.head()

,affy_hg_u133_plus_2,hgnc_symbol,gene_biotype,GSM1130907_Joo091-HG-U133Plus2.cel.gz,GSM1130908_Joo092-HG-U133Plus2.cel.gz,GSM1130909_Joo093-HG-U133Plus2.cel.gz,GSM1130910_Joo094-HG-U133Plus2.cel.gz,GSM1130911_Joo095-HG-U133Plus2.cel.gz,GSM1130912_Joo096-HG-U133Plus2.cel.gz,GSM1130913_Joo097-HG-U133Plus2.cel.gz,...,GSM1130953_Joo148-HG-U133Plus2.cel.gz,GSM1130954_Joo149-HG-U133Plus2.cel.gz,GSM1130955_Joo150-HG-U133Plus2.cel.gz,GSM1130956_Joo151-HG-U133Plus2.cel.gz,GSM1130957_Joo152-HG-U133Plus2.cel.gz,GSM1130958_Joo153-HG-U133Plus2.cel.gz,GSM1130959_Joo154-HG-U133Plus2.cel.gz,GSM1130960_Joo155-HG-U133Plus2.cel.gz,GSM1130961_Joo156-HG-U133Plus2.cel.gz,label
0,215559_at,ABCC6,protein_coding,3.387972,3.197034,3.066003,3.270180,3.483682,3.487162,2.901785,...,3.575743,3.382958,3.418691,3.125065,3.249330,3.413010,3.325482,3.458992,3.289809,1
1,214033_at,ABCC6,protein_coding,4.991891,5.131626,4.714632,4.880840,4.844942,4.833550,4.635570,...,5.068621,4.789166,4.754656,4.859572,4.854206,4.946681,4.862422,5.012400,4.610862,1
2,208480_s_at,ABCC6,protein_coding,4.204203,4.254020,4.781491,4.468411,4.432407,4.249169,4.588663,...,4.237186,4.442878,4.556832,4.827632,4.286801,4.573060,4.598085,4.873792,4.735709,1
3,207427_at,ACR,protein_coding,5.310477,5.301529,5.402414,5.248901,5.335763,5.170322,5.697215,...,5.471443,5.730578,5.318175,5.083529,5.393456,5.319998,5.289243,5.454217,5.741971,0
4,AFFX-HSAC07/X00351_M_at,ACTB,protein_coding,12.456092,11.877125,11.971389,12.726054,12.703934,12.901557,11.662964,...,12.796530,12.477227,12.732850,11.979345,12.830888,12.722848,12.252823,11.755974,12.576457,1


In [134]:
train = train.drop(['affy_hg_u133_plus_2'], axis=1)
test = test.drop(['affy_hg_u133_plus_2'], axis=1)
#TODO Consultar si se tiene que quitar gene_biotype

In [135]:
train.head()

,hgnc_symbol,gene_biotype,GSM1130907_Joo091-HG-U133Plus2.cel.gz,GSM1130908_Joo092-HG-U133Plus2.cel.gz,GSM1130909_Joo093-HG-U133Plus2.cel.gz,GSM1130910_Joo094-HG-U133Plus2.cel.gz,GSM1130911_Joo095-HG-U133Plus2.cel.gz,GSM1130912_Joo096-HG-U133Plus2.cel.gz,GSM1130913_Joo097-HG-U133Plus2.cel.gz,GSM1130914_Joo098-HG-U133Plus2.cel.gz,...,GSM1130953_Joo148-HG-U133Plus2.cel.gz,GSM1130954_Joo149-HG-U133Plus2.cel.gz,GSM1130955_Joo150-HG-U133Plus2.cel.gz,GSM1130956_Joo151-HG-U133Plus2.cel.gz,GSM1130957_Joo152-HG-U133Plus2.cel.gz,GSM1130958_Joo153-HG-U133Plus2.cel.gz,GSM1130959_Joo154-HG-U133Plus2.cel.gz,GSM1130960_Joo155-HG-U133Plus2.cel.gz,GSM1130961_Joo156-HG-U133Plus2.cel.gz,label
0,ABCC6,protein_coding,3.387972,3.197034,3.066003,3.270180,3.483682,3.487162,2.901785,3.138349,...,3.575743,3.382958,3.418691,3.125065,3.249330,3.413010,3.325482,3.458992,3.289809,1
1,ABCC6,protein_coding,4.991891,5.131626,4.714632,4.880840,4.844942,4.833550,4.635570,4.690146,...,5.068621,4.789166,4.754656,4.859572,4.854206,4.946681,4.862422,5.012400,4.610862,1
2,ABCC6,protein_coding,4.204203,4.254020,4.781491,4.468411,4.432407,4.249169,4.588663,4.673135,...,4.237186,4.442878,4.556832,4.827632,4.286801,4.573060,4.598085,4.873792,4.735709,1
3,ACR,protein_coding,5.310477,5.301529,5.402414,5.248901,5.335763,5.170322,5.697215,5.669347,...,5.471443,5.730578,5.318175,5.083529,5.393456,5.319998,5.289243,5.454217,5.741971,0
4,ACTB,protein_coding,12.456092,11.877125,11.971389,12.726054,12.703934,12.901557,11.662964,11.376700,...,12.796530,12.477227,12.732850,11.979345,12.830888,12.722848,12.252823,11.755974,12.576457,1


In [136]:
train.columns

Index(['hgnc_symbol', 'gene_biotype', 'GSM1130907_Joo091-HG-U133Plus2.cel.gz',
       'GSM1130908_Joo092-HG-U133Plus2.cel.gz',
       'GSM1130909_Joo093-HG-U133Plus2.cel.gz',
       'GSM1130910_Joo094-HG-U133Plus2.cel.gz',
       'GSM1130911_Joo095-HG-U133Plus2.cel.gz',
       'GSM1130912_Joo096-HG-U133Plus2.cel.gz',
       'GSM1130913_Joo097-HG-U133Plus2.cel.gz',
       'GSM1130914_Joo098-HG-U133Plus2.cel.gz',
       'GSM1130915_Joo099-HG-U133Plus2.cel.gz',
       'GSM1130916_Joo100-HG-U133Plus2.cel.gz',
       'GSM1130917_Joo101-HG-U133Plus2.cel.gz',
       'GSM1130918_Joo102-HG-U133Plus2.cel.gz',
       'GSM1130919_Joo109-HG-U133Plus2.cel.gz',
       'GSM1130920_Joo110-HG-U133Plus2.cel.gz',
       'GSM1130921_Joo111-HG-U133Plus2.cel.gz',
       'GSM1130922_Joo113-HG-U133Plus2.cel.gz',
       'GSM1130923_Joo114-HG-U133Plus2.cel.gz',
       'GSM1130924_Joo115-HG-U133Plus2.cel.gz',
       'GSM1130925_Joo116-HG-U133Plus2.cel.gz',
       'GSM1130926_Joo118-HG-U133Plus2.cel.gz',
       'G

In [137]:
train.shape

(944, 58)

In [138]:
train.nunique()

hgnc_symbol                              376
gene_biotype                               3
GSM1130907_Joo091-HG-U133Plus2.cel.gz    864
GSM1130908_Joo092-HG-U133Plus2.cel.gz    872
GSM1130909_Joo093-HG-U133Plus2.cel.gz    872
GSM1130910_Joo094-HG-U133Plus2.cel.gz    872
GSM1130911_Joo095-HG-U133Plus2.cel.gz    875
GSM1130912_Joo096-HG-U133Plus2.cel.gz    864
GSM1130913_Joo097-HG-U133Plus2.cel.gz    867
GSM1130914_Joo098-HG-U133Plus2.cel.gz    873
GSM1130915_Joo099-HG-U133Plus2.cel.gz    871
GSM1130916_Joo100-HG-U133Plus2.cel.gz    870
GSM1130917_Joo101-HG-U133Plus2.cel.gz    868
GSM1130918_Joo102-HG-U133Plus2.cel.gz    865
GSM1130919_Joo109-HG-U133Plus2.cel.gz    869
GSM1130920_Joo110-HG-U133Plus2.cel.gz    870
GSM1130921_Joo111-HG-U133Plus2.cel.gz    860
GSM1130922_Joo113-HG-U133Plus2.cel.gz    863
GSM1130923_Joo114-HG-U133Plus2.cel.gz    862
GSM1130924_Joo115-HG-U133Plus2.cel.gz    870
GSM1130925_Joo116-HG-U133Plus2.cel.gz    871
GSM1130926_Joo118-HG-U133Plus2.cel.gz    876
GSM1130927

In [139]:
train.gene_biotype.value_counts()


protein_coding    936
lncRNA              4
miRNA               4
Name: gene_biotype, dtype: int64

In [140]:
#Solo se utilizaran los LncRNA
test.gene_biotype.value_counts()

protein_coding    37270
lncRNA             3052
miRNA               176
snoRNA              111
IG_V_gene            47
TR_V_gene            38
snRNA                33
IG_C_gene            29
TR_C_gene            18
misc_RNA             12
TEC                  10
Mt_tRNA               8
scaRNA                5
IG_J_gene             4
rRNA                  2
ribozyme              1
Name: gene_biotype, dtype: int64

In [141]:
test = test[test.gene_biotype == 'lncRNA']
test.gene_biotype.value_counts()

lncRNA    3052
Name: gene_biotype, dtype: int64

In [142]:
train['label'].value_counts()

1    590
0    354
Name: label, dtype: int64

# Encoding


In [143]:
le = LabelEncoder()
train['hgnc_symbol'] = le.fit_transform(train['hgnc_symbol'])
train['gene_biotype'] = le.fit_transform(train['gene_biotype'])
train.head()

,hgnc_symbol,gene_biotype,GSM1130907_Joo091-HG-U133Plus2.cel.gz,GSM1130908_Joo092-HG-U133Plus2.cel.gz,GSM1130909_Joo093-HG-U133Plus2.cel.gz,GSM1130910_Joo094-HG-U133Plus2.cel.gz,GSM1130911_Joo095-HG-U133Plus2.cel.gz,GSM1130912_Joo096-HG-U133Plus2.cel.gz,GSM1130913_Joo097-HG-U133Plus2.cel.gz,GSM1130914_Joo098-HG-U133Plus2.cel.gz,...,GSM1130953_Joo148-HG-U133Plus2.cel.gz,GSM1130954_Joo149-HG-U133Plus2.cel.gz,GSM1130955_Joo150-HG-U133Plus2.cel.gz,GSM1130956_Joo151-HG-U133Plus2.cel.gz,GSM1130957_Joo152-HG-U133Plus2.cel.gz,GSM1130958_Joo153-HG-U133Plus2.cel.gz,GSM1130959_Joo154-HG-U133Plus2.cel.gz,GSM1130960_Joo155-HG-U133Plus2.cel.gz,GSM1130961_Joo156-HG-U133Plus2.cel.gz,label
0,0,2,3.387972,3.197034,3.066003,3.270180,3.483682,3.487162,2.901785,3.138349,...,3.575743,3.382958,3.418691,3.125065,3.249330,3.413010,3.325482,3.458992,3.289809,1
1,0,2,4.991891,5.131626,4.714632,4.880840,4.844942,4.833550,4.635570,4.690146,...,5.068621,4.789166,4.754656,4.859572,4.854206,4.946681,4.862422,5.012400,4.610862,1
2,0,2,4.204203,4.254020,4.781491,4.468411,4.432407,4.249169,4.588663,4.673135,...,4.237186,4.442878,4.556832,4.827632,4.286801,4.573060,4.598085,4.873792,4.735709,1
3,1,2,5.310477,5.301529,5.402414,5.248901,5.335763,5.170322,5.697215,5.669347,...,5.471443,5.730578,5.318175,5.083529,5.393456,5.319998,5.289243,5.454217,5.741971,0
4,2,2,12.456092,11.877125,11.971389,12.726054,12.703934,12.901557,11.662964,11.376700,...,12.796530,12.477227,12.732850,11.979345,12.830888,12.722848,12.252823,11.755974,12.576457,1


In [144]:
le = LabelEncoder()
test['hgnc_symbol'] = le.fit_transform(test['hgnc_symbol'])
test['gene_biotype'] = le.fit_transform(test['gene_biotype'])
test.head()

,hgnc_symbol,gene_biotype,GSM1130907_Joo091-HG-U133Plus2.cel.gz,GSM1130908_Joo092-HG-U133Plus2.cel.gz,GSM1130909_Joo093-HG-U133Plus2.cel.gz,GSM1130910_Joo094-HG-U133Plus2.cel.gz,GSM1130911_Joo095-HG-U133Plus2.cel.gz,GSM1130912_Joo096-HG-U133Plus2.cel.gz,GSM1130913_Joo097-HG-U133Plus2.cel.gz,GSM1130914_Joo098-HG-U133Plus2.cel.gz,...,GSM1130952_Joo147-HG-U133Plus2.cel.gz,GSM1130953_Joo148-HG-U133Plus2.cel.gz,GSM1130954_Joo149-HG-U133Plus2.cel.gz,GSM1130955_Joo150-HG-U133Plus2.cel.gz,GSM1130956_Joo151-HG-U133Plus2.cel.gz,GSM1130957_Joo152-HG-U133Plus2.cel.gz,GSM1130958_Joo153-HG-U133Plus2.cel.gz,GSM1130959_Joo154-HG-U133Plus2.cel.gz,GSM1130960_Joo155-HG-U133Plus2.cel.gz,GSM1130961_Joo156-HG-U133Plus2.cel.gz
43,114,0,3.470322,3.248116,3.108233,3.111788,3.177630,2.865521,3.376438,3.344414,...,3.308748,3.205522,3.362456,3.387258,3.194051,3.109982,3.329530,3.569103,3.393265,3.462450
47,150,0,4.189275,4.002933,4.146277,4.011822,4.089458,3.978372,3.865671,3.992745,...,3.945360,4.054124,4.380124,4.171883,3.914398,4.230883,3.990811,3.920596,4.185160,4.116805
56,615,0,2.505303,2.416484,2.410177,2.069271,2.306033,2.344725,2.482299,2.522221,...,2.474032,2.276189,2.323617,2.363810,2.095357,2.056123,2.264082,2.454293,2.177812,2.443720
57,622,0,3.286622,2.524011,2.768894,3.367125,3.618483,3.166733,2.911684,3.041330,...,2.806335,3.756825,2.777507,3.919810,2.587067,3.052068,2.610142,3.001631,3.090771,3.462450
58,622,0,2.557472,2.431744,2.270320,2.491757,2.487217,2.655941,2.512167,2.556472,...,2.508927,2.424956,2.457619,2.393461,2.429525,2.403210,2.512043,2.586467,2.601225,2.564901


# Split del dataset

In [145]:
train_x = train.drop(columns=['label'])
train_y = train['label']
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size= 0.2, random_state = 1)


Sensitivity formula $$ \frac{TP}{TP+FN}$$

# Creación y evaluacion del modelo 

In [95]:
rfc = RandomForestClassifier(n_estimators= 100, max_features= 'auto', max_depth=20, random_state=0)
rfc.fit(train_x, train_y)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(rfc, test_x, test_y, scoring='recall', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Recall: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Recall: 0.868 (0.094)


# Grid Search

In [13]:
hiperpararmetros = {'max_depth': [5,10,15],
                    'max_features': ['auto','log2','sqrt'],
                    'n_estimators': [50, 100, 200]}

grid_search = GridSearchCV(RandomForestClassifier(), hiperpararmetros, cv=10, scoring= 'accuracy')
grid_search.fit(train_x, train_y)

print("Test set score: {:.2f}".format(grid_search.score(test_x, test_y)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

Test set score: 0.91
Best parameters: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 50}
Best cross-validation score: 0.92


# Report

In [113]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size= 0.2, random_state = 1)
tuned_model = RandomForestClassifier(  max_depth=5, max_features= 'log2', n_estimators= 50, random_state=0)
tuned_model.fit(train_x, train_y)


RandomForestClassifier(max_depth=5, max_features='log2', n_estimators=50,
                       random_state=0)

In [114]:
y_pred_train = tuned_model.predict(train_x)
print(classification_report(train_y, y_pred_train))

              precision    recall  f1-score   support

           0       0.98      0.59      0.74        91
           1       0.81      0.99      0.89       155

    accuracy                           0.85       246
   macro avg       0.89      0.79      0.81       246
weighted avg       0.87      0.85      0.83       246



In [115]:
y_pred_test = tuned_model.predict(test_x)
print(classification_report(test_y, y_pred_test))

              precision    recall  f1-score   support

           0       0.56      0.20      0.29        25
           1       0.62      0.89      0.73        37

    accuracy                           0.61        62
   macro avg       0.59      0.55      0.51        62
weighted avg       0.60      0.61      0.56        62



# Grid Search (n_estimators)


In [158]:
hiperpararmetros = {'max_depth': [5,10,15],
                    'max_features': ['auto','log2','sqrt'],
                    'n_estimators': [250, 500, 750]}

grid_search = GridSearchCV(RandomForestClassifier(), hiperpararmetros, cv=10, scoring= 'accuracy')
grid_search.fit(train_x, train_y)

print("Test set score: {:.2f}".format(grid_search.score(test_x, test_y)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

Test set score: 0.69
Best parameters: {'max_depth': 10, 'max_features': 'auto', 'n_estimators': 500}
Best cross-validation score: 0.66


# Report (n_estimators)

In [159]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size= 0.2, random_state = 420)
tuned_model = RandomForestClassifier(  max_depth=5, max_features= 'log2', n_estimators= 500, random_state=69)
tuned_model.fit(train_x, train_y)

RandomForestClassifier(max_depth=5, max_features='log2', n_estimators=500,
                       random_state=69)

In [160]:
y_pred_train = tuned_model.predict(train_x)
print(classification_report(train_y, y_pred_train))

              precision    recall  f1-score   support

           0       0.93      0.46      0.61       235
           1       0.74      0.98      0.84       369

    accuracy                           0.77       604
   macro avg       0.83      0.72      0.73       604
weighted avg       0.81      0.77      0.75       604



In [161]:
y_pred_test = tuned_model.predict(test_x)
print(classification_report(test_y, y_pred_test))

              precision    recall  f1-score   support

           0       0.75      0.16      0.26        58
           1       0.65      0.97      0.78        93

    accuracy                           0.66       151
   macro avg       0.70      0.56      0.52       151
weighted avg       0.69      0.66      0.58       151



# Smote

In [119]:
oversample = SMOTE()

train_x = train.drop(columns=['label'])
train_y = train['label']
smoted_x, test_x, smoted_y, test_y = train_test_split(train_x, train_y, test_size= 0.2, random_state = 1)


smoted_x, smoted_y = oversample.fit_resample(smoted_x, smoted_y)

In [120]:
smoted_y.value_counts()

0    462
1    462
Name: label, dtype: int64

# Grid Search (SMOTE)


In [122]:
hiperpararmetros = {'max_depth': [5,10,15],
                    'max_features': ['auto','log2','sqrt'],
                    'n_estimators': [250, 500, 750]}

grid_search = GridSearchCV(RandomForestClassifier(), hiperpararmetros, cv=10, scoring= 'accuracy')
grid_result = grid_search.fit(smoted_x, smoted_y)

print("Test set score: {:.2f}".format(grid_search.score(test_x, test_y)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

Test set score: 0.65
Best parameters: {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 250}
Best cross-validation score: 0.73


In [123]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.731755 using {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 250}
0.658111 (0.055295) with: {'max_depth': 5, 'max_features': 'auto', 'n_estimators': 250}
0.664610 (0.058165) with: {'max_depth': 5, 'max_features': 'auto', 'n_estimators': 500}
0.668957 (0.061726) with: {'max_depth': 5, 'max_features': 'auto', 'n_estimators': 750}
0.635437 (0.069282) with: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 250}
0.650561 (0.061779) with: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 500}
0.654885 (0.059844) with: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 750}
0.653857 (0.068474) with: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 250}
0.667882 (0.061565) with: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 500}
0.657048 (0.061818) with: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 750}
0.700386 (0.056600) with: {'max_depth': 10, 'max_features': 'auto', 'n_estimators': 250}
0.701461 (0.063417) with: {'max_dep

# Report (SMOTE)

In [155]:

tuned_model = RandomForestClassifier(  max_depth=15, max_features= 'auto', n_estimators= 250, random_state=1)
tuned_model.fit(smoted_x, smoted_y)

RandomForestClassifier(max_depth=15, n_estimators=250, random_state=1)

In [156]:
y_pred_smoted = tuned_model.predict(smoted_x)
print(classification_report(smoted_y, y_pred_smoted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       462
           1       1.00      1.00      1.00       462

    accuracy                           1.00       924
   macro avg       1.00      1.00      1.00       924
weighted avg       1.00      1.00      1.00       924



In [157]:
y_pred_test = tuned_model.predict(test_x)
print(classification_report(test_y, y_pred_test))

              precision    recall  f1-score   support

           0       0.49      0.62      0.55        61
           1       0.79      0.70      0.74       128

    accuracy                           0.67       189
   macro avg       0.64      0.66      0.65       189
weighted avg       0.70      0.67      0.68       189



# Optimization 

In [ ]:
hiperpararmetros = {'max_depth': [5,10,15,20,25,30,35,40],
                    'max_features': ['auto','log2','sqrt'],
                    'n_estimators': [100,200,300,400,500,600,700,800]}

grid_search = GridSearchCV(RandomForestClassifier(), hiperpararmetros, cv=10, scoring= 'accuracy')
grid_result = grid_search.fit(smoted_x, smoted_y)

print("Test set score: {:.2f}".format(grid_search.score(test_x, test_y)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))